<a href="https://colab.research.google.com/github/bbutka/CEC220/blob/main/Hamming_Distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hamming_Distance
© 2024 Brian Butka. All rights reserved.

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import itertools

class HammingDistanceCalculator:
    def __init__(self):
        self.num_symbols = 0
        self.symbols = []
        self.symbol_inputs = []
        self.init_ui()

    def init_ui(self):
        self.num_symbols_label = widgets.Label(
            value='Number of symbols:'
        )

        self.num_symbols_input = widgets.IntText(
            value=3,
            disabled=False
        )

        self.submit_symbols_button = widgets.Button(
            description='Submit',
            disabled=False,
            button_style='',
            tooltip='Click to submit',
            icon='check'
        )
        self.submit_symbols_button.on_click(self.on_submit_symbols)

        self.symbols_form = widgets.VBox()

        self.submit_symbol_button = widgets.Button(
            description='Calculate Distances',
            disabled=False,
            button_style='',
            tooltip='Click to calculate',
            icon='check'
        )
        self.submit_symbol_button.on_click(self.on_submit_symbol)

        self.restart_button = widgets.Button(
            description='Restart',
            disabled=False,
            button_style='',
            tooltip='Click to restart',
            icon='refresh'
        )
        self.restart_button.on_click(self.on_restart)

        display(widgets.VBox([
            widgets.HBox([self.num_symbols_label, self.num_symbols_input]),
            self.submit_symbols_button
        ]))

    def on_submit_symbols(self, button):
        self.num_symbols = self.num_symbols_input.value
        self.symbols = []
        self.symbol_inputs = []

        symbol_widgets = []
        for i in range(self.num_symbols):
            symbol_input = widgets.Text(
                value='',
                placeholder=f'Enter binary or hex symbol {i+1} (e.g., 1010 or 0xA)',
                description=f'Symbol {i+1}:',
                layout=widgets.Layout(width='400px')  # Increase width of text input
            )
            self.symbol_inputs.append(symbol_input)
            symbol_widgets.append(symbol_input)

        self.symbols_form.children = symbol_widgets
        clear_output()
        display(widgets.VBox([self.symbols_form, self.submit_symbol_button]))

    def on_submit_symbol(self, button):
        clear_output()

        first_symbol_length = None
        for symbol_input in self.symbol_inputs:
            symbol = symbol_input.value.strip().lower()

            # Detect if the symbol is in hexadecimal (starts with '0x')
            if symbol.startswith('0x'):
                try:
                    # Convert hex to binary, removing the '0x' prefix
                    normalized_symbol = bin(int(symbol, 16))[2:]
                    # Ensure the binary string is padded to the correct length
                    if first_symbol_length:
                        normalized_symbol = normalized_symbol.zfill(first_symbol_length)
                except ValueError:
                    print(f"Error: Invalid hexadecimal input '{symbol}'.")
                    display(widgets.VBox([self.symbols_form, self.submit_symbol_button, self.restart_button]))
                    return
            else:
                # Assume the input is binary; check if valid binary input
                if not all(c in '01' for c in symbol):
                    print(f"Error: Invalid binary input '{symbol}'.")
                    display(widgets.VBox([self.symbols_form, self.submit_symbol_button, self.restart_button]))
                    return
                normalized_symbol = symbol

            if first_symbol_length is None:
                first_symbol_length = len(normalized_symbol)

            self.symbols.append(normalized_symbol.zfill(first_symbol_length))

        # Check if all symbols have the same length
        if any(len(symbol) != first_symbol_length for symbol in self.symbols):
            print(f"Error: All symbols must be {first_symbol_length} bits long.")
            display(widgets.VBox([self.symbols_form, self.submit_symbol_button, self.restart_button]))
            return

        distances = self.calculate_hamming_distances(self.symbols)
        self.display_hamming_distances(distances, self.symbols)

        display(self.restart_button)

    def on_restart(self, button):
        clear_output()
        self.init_ui()

    @staticmethod
    def hamming_distance(s1, s2):
        return sum(c1 != c2 for c1, c2 in zip(s1, s2))

    def calculate_hamming_distances(self, symbols):
        distances = []
        for (i, s1), (j, s2) in itertools.combinations(enumerate(symbols), 2):
            distance = self.hamming_distance(s1, s2)
            xor_result = ''.join('<span style="color:red;">1</span>' if c1 != c2 else '0' for c1, c2 in zip(s1, s2))
            distances.append((i, j, distance, xor_result))
        return distances

    def display_hamming_distances(self, distances, symbols):
        print("\nPairwise Hamming Distances:")
        for i, j, distance, xor_result in distances:
            display(HTML(f"""
            <div>Symbols {i+1} and {j+1}:</div>
            <div>{symbols[i]}</div>
            <div>{symbols[j]}</div>
            <div>{xor_result}</div>
            <div>Hamming Distance: {distance}</div>
            """))

        min_distance = min(distance for _, _, distance, _ in distances)
        print(f"\nMinimum Hamming Distance for the system: {min_distance}")

        # Add the error detection and correction section
        self.display_error_detection_and_correction(min_distance)

    def display_error_detection_and_correction(self, min_distance):
        # Table for detection and correction based on Hamming Distance
        hamming_table = {
            1: (0, 0),
            2: (1, 0),
            3: (1, 1),
            4: (2, 1),
            5: (2, 2),
            6: (3, 2),
            7: (3, 3),
            8: (4, 3),
            9: (4, 4)
        }

        if min_distance in hamming_table:
            error_detection, error_correction = hamming_table[min_distance]
            print(f"\nError Detection and Correction Capabilities:")
            print(f"A Hamming distance of {min_distance} can detect up to {error_detection} errors.")
            print(f"A Hamming distance of {min_distance} can correct up to {error_correction} errors.")
        else:
            print(f"Error: Hamming distance {min_distance} is out of the table's supported range.")

HammingDistanceCalculator()



Pairwise Hamming Distances:



Minimum Hamming Distance for the system: 4

Error Detection and Correction Capabilities:
A Hamming distance of 4 can detect up to 2 errors.
A Hamming distance of 4 can correct up to 1 errors.


Button(description='Restart', icon='refresh', style=ButtonStyle(), tooltip='Click to restart')